# 뉴스 자동 수집기 만들기 (`selenium` 사용)

키워드와 기간이 주어지면 키워드에 해당하는 기간 내 뉴스 검색 결과를 자동으로 수집하는 도구를 만들어보자.

# 스크롤 수만큼 페이지 로딩 후 뉴스 스크랩

In [ ]:
def scroll_down(driver, num_scroll=-1):
    """
    페이지 스크롤 다운 (무한 스크롤: num_scroll=-1 입력)
    """
    #스크롤 내리기 이동 전 위치
    old_location = driver.execute_script("return document.body.scrollHeight")
    scroll_count = 0
    
    while True:
    	#현재 스크롤의 가장 아래로 내림
        driver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        #전체 스크롤이 늘어날 때까지 대기
        time.sleep(2)
        #늘어난 스크롤 높이
        new_location = driver.execute_script("return document.body.scrollHeight")
        print('scroll down..')
        #늘어난 스크롤 위치와 이동 전 위치 같으면(더 이상 스크롤이 늘어나지 않으면) 종료
        if new_location == old_location or scroll_count == num_scroll:
        	break
        #스크롤 위치값을 수정
        old_location = new_location
        scroll_count += 1

# 검색 결과 페이지에서 뉴스 메타 정보 수집

In [ ]:
def list_up_news(driver):
    
    titles, presses, links = [], [], []
    title_selector = 'div.news_wrap.api_ani_send > div > div.news_contents > a.news_tit'
    press_selector = 'div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a.info.press'

    for i in driver.find_elements('css selector', title_selector):
        presses.append(i.text.replace('언론사 선정', ''))
    
    for i in driver.find_elements('css selector', title_selector):
        titles.append(i.text)
        links.append(i.get_attribute("href"))
    
    return titles, presses, links

# 뉴스 url마다 접속해서 뉴스 본문 수집

In [ ]:
from newspaper import ArticleException

def collect_news_text(links):
    texts = []
    
    for link in links:
        print('기사 url :', link)
        try:
            # 언어를 한국어로 설정하고 URL을 전달해 Article 클래스의 객체 생성
            article = Article(link, language='ko')
            # 지정된 웹 페이지를 다운로드
            article.download()
            # 다운로드한 웹 페이지를 분석하고 필요한 정보를 추출
            article.parse()
            text = article.text
            print('기사 내용 :', text[:100].replace('\n', ''))
            print('')
        except ArticleException:
            text = None
            print('기사 내용 : 에러\n')
            print('')
        
        texts.append(text)
        time.sleep(1)
    return texts

# 활용해보기

In [ ]:
from selenium import webdriver
from newspaper import Article
import time

# 키워드, URL
keyword = '비트코인'
ds = '2025.03.01'
de = '2025.03.02'
url = f'https://search.naver.com/search.naver?where=news&query={keyword}&pd=3&ds={ds}&de={de}'

# driver
driver = webdriver.Chrome()

# get page
driver.get(url)
driver.implicitly_wait(2) # 2초 안에 웹페이지를 load하면 바로 넘어가거나, 2초를 기다림

# scroll down
scroll_down(driver, num_scroll=3)

In [ ]:
titles, presses, links = list_up_news(driver)

print("가져온 뉴스 수: ", len(titles))
print('')

for title, press, link in zip(titles, presses, links):
    print("제목: ", title)
    print("언론사: ", press)
    print("주소: ", link)
    print()


In [ ]:
texts = collect_news_text(links)

In [ ]:
# don't forget to close the driver!
driver.close()